In [1]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/marcin/.julia/compiled/v0.7/AcausalNets/sM5bh.ji for AcausalNets [f45f996a-9c6d-58c4-b63c-9e6370055198]
└ @ Base loading.jl:1185
┌ Warning: Package AcausalNets does not have QI in its dependencies:
│ - If you have AcausalNets checked out for development and have
│   added QI as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AcausalNets
└ Loading QI into AcausalNets from project dependency, future warnings for AcausalNets are suppressed.


In [2]:
roA = Diagonal([1/3, 1/3, 1/3])

roB = Diagonal([1/3, 1/3, 1/3])
roCwAB = Diagonal([
        0,1/2, 1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        #CHANGED to that ro(C | AB) != ro(C | BA) - Originally BE 0,1,0
        0,1,0, #A=0, B=2 
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_a = DiscreteQuantumSystem([var_a], roA)

sys_b = DiscreteQuantumSystem([var_b], roB)

sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
classic_an = AcausalNet()

push!(classic_an, sys_a)
push!(classic_an, sys_b)
push!(classic_an, sys_c_ab)
# show(an)

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({3, 2} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 3)], [0.333333 0.0 0.0; 0.0 0.333333 0.0; 0.0 0.0 0.333333]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:b, 3)], [0.333333 0.0 0.0; 0.0 0.333333 0.0; 0.0 0.0 0.333333]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 3), Variable(:b, 3)], Variable[Variable(:c, 3)], [0.0 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.0; 0.0 0.0 … 0.0 0.0])])

In [3]:
roAB =1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 

roC = Diagonal([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_ab = DiscreteQuantumSystem([var_a, var_b], roAB)
sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
quantum_an = AcausalNet()

push!(quantum_an, sys_ab)
push!(quantum_an, sys_c_ab)

# gplot(Graph(quantum_bn.dag), nodelabel=nodes(quantum_bn))

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({2, 1} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 3), Variable(:b, 3)], Complex{Float64}[0.333333+0.0im 0.0+0.0im … 0.0+0.0im 0.333333+0.0im; 0.0+0.0im 0.0+0.0im … 0.0+0.0im 0.0+0.0im; … ; 0.0+0.0im 0.0+0.0im … 0.0+0.0im 0.0+0.0im; 0.333333+0.0im 0.0+0.0im … 0.0+0.0im 0.333333+0.0im]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 3), Variable(:b, 3)], Variable[Variable(:c, 3)], [0.0 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.5 0.0; 0.0 0.0 … 0.0 0.0])])

In [4]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]


AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({8, 9} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 2)], [0.5 0.0; 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:b, 2)], [0.5 0.0 0.0 0.0; 0.0 0.5 0.0 0.0; 0.0 0.0 0.4 0.0; 0.0 0.0 0.0 0.6]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:b, 2)], Variable[Variable(:d, 2)], [0.9 0.0 0.0 0.0; 0.0 0.1 0.0 0.0; 0.0 0.0 0.5 0.0; 0.0 0.0 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{Abstrac

In [5]:
# roA = Diagonal([.5, .5])

# roBwA = Diagonal([.5, .5, .5, .5])

# roCwB = Diagonal([.5, .5, .5, .5])

# roDwC = Diagonal([.5, .5, .5, .5])
# |
# sys_a = DiscreteQuantumSystem([var_a], roA)
# sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
# sys_c = DiscreteQuantumSystem([var_b], [var_c], roCwB)
# sys_d = DiscreteQuantumSystem([var_c], [var_d], roDwC)

# snake_an = AcausalNet()
# push!(snake_an, sys_a)
# push!(snake_an, sys_b)
# push!(snake_an, sys_c)
# push!(snake_an, sys_d)

# # gplot(snake_an.dag, nodelabel=nodes(snake_bn))

In [6]:
function moral_graph(an::AcausalNet)
    result = Graph(deepcopy(an.dag))
    for node in an.systems
        for p1 in node.parents
            for p2 in node.parents
                p1_ind = variable_to_node(an, p1)
                p2_ind = variable_to_node(an, p2)
                if(p1_ind!=p2_ind)
                    add_edge!(result, p1_ind, p2_ind)
                end
            end
        end
    end
    return result
end

moral_graph (generic function with 1 method)

In [7]:
an = example_an
moral_an = moral_graph(an)
print(ne(moral_an))
# gplot(moral_bn, nodelabel=nodes(bn))
println(moral_an.fadjlist)
println(variables_names(an))

11Array{Int64,1}[[2, 3], [1, 4], [1, 5, 7], [2, 5, 6], [3, 4, 6, 7, 8], [4, 5], [3, 5, 8], [5, 7]]
Symbol[:a, :b, :c, :d, :e, :f, :g, :h]


In [8]:
is_subset(s1::Set, s2::Set) = intersect(s1, s2) == s1

is_subset (generic function with 1 method)

In [9]:
function triangulate(mg::Graph, an::AcausalNet)
    mg_copy = [false for _ in vertices(mg)]
    mg = deepcopy(mg)
    nl = systems(an)
    cliques = Vector{Set}()
    while(!all(mg_copy))
        least_edges_to_be_added = Inf
        chosen_vertex = 0
        chosen_cluster = Set()
        for v=1:length(mg_copy)
            if mg_copy[v]
                continue
            else
                cluster = Set()
                for e in edges(mg)
                    if (v==src(e) || v==dst(e)) && !mg_copy[src(e)] && !mg_copy[dst(e)]
                        push!(cluster, src(e))
                        push!(cluster, dst(e))
                    end
                end
                edges_todo = 0
                for v1 in cluster
                    for v2 in cluster
                        if v1 != v2 && !in((v1, v2), edges(mg))
                            edges_todo +=1
                        end
                    end
                end
                edges_todo /= 2
                
                if edges_todo < least_edges_to_be_added || 
                    ((edges_todo == least_edges_to_be_added) && 
                        (prod([ncategories(systems(an)[n]) for n in cluster]) <= prod([ncategories(systems(an)[n]) for n in chosen_cluster])))                      
                    least_edges_to_be_added = edges_todo
                    chosen_vertex = v
                    chosen_cluster = cluster
                end
            end       
        end
        chosen_nodes = Set([nl[n] for n in chosen_cluster])
        if !any([is_subset(chosen_nodes, clique) for clique in cliques])
            push!(cliques, chosen_nodes)
        end
        mg_copy[chosen_vertex] = true
        for v1 in chosen_cluster
            for v2 in chosen_cluster
                if v1 != v2 && !in((v1, v2), edges(mg))
                    add_edge!(mg, v1, v2)
                end
            end
        end     
    end
    cliques = [sort([c for c in clique], by=c -> system_to_node(an, c)) for clique in cliques]
    return mg, cliques
end

triangulate (generic function with 1 method)

In [10]:
println(ne(moral_an))

tri_moral_an, cliques = triangulate(moral_an,an)
println(ne(tri_moral_an))

# before, cliques were sets
# they are now transformed to vectors and their ordering is the same 
# as in their bayes_net DAG
# println(cliques)
# gplot(tri_moral_an ,nodelabel=nodes(an))

println(tri_moral_an.fadjlist)
println(variables_names(an))

11


┌ Warning: `findfirst(A, v)` is deprecated, use `something(findfirst(isequal(v), A), 0)` instead.
│   caller = system_to_node at bayes_net.jl:30 [inlined]
└ @ Core /home/marcin/.julia/dev/AcausalNets/src/Structures/bayes_net.jl:30
┌ Warning: `findfirst(A, v)` is deprecated, use `something(findfirst(isequal(v), A), 0)` instead.
│   caller = system_to_node at bayes_net.jl:30 [inlined]
└ @ Core /home/marcin/.julia/dev/AcausalNets/src/Structures/bayes_net.jl:30


13
Array{Int64,1}[[2, 3], [1, 3, 4], [1, 2, 4, 5, 7], [2, 3, 5, 6], [3, 4, 6, 7, 8], [4, 5], [3, 5, 8], [5, 7]]
Symbol[:a, :b, :c, :d, :e, :f, :g, :h]


In [11]:
struct JoinTree
    graph::Graph
    clusters::Vector{Vector{DiscreteQuantumSystem}}
    vertex_to_num::Dict{Int64, Union{Float64, Complex, Matrix}}
    edge_to_num::Dict{Set{Int}, Union{Float64, Complex, Matrix}}
    
end

In [12]:
function sepset_cost(sepset::Set{DiscreteQuantumSystem}, an::AcausalNet)
    weights = [ncategories(s) for s in sepset]
    
    return 0
end

sepset_cost (generic function with 1 method)

In [13]:
function sepset_comparator(c1, c2)
    sepset = intersect(c1, c2)
    -length(sepset)
end

sepset_comparator (generic function with 1 method)

In [14]:
cluster_size(cluster::Vector{DiscreteQuantumSystem}, an::AcausalNet) = prod([ncategories(s) for s in cluster])

cluster_size (generic function with 1 method)

In [15]:
function make_join_tree(clusters::Vector{Vector{DiscreteQuantumSystem}}, an::AcausalNet)
    candidate_sepsets = []
    trees = Dict([c => c for c in clusters])
    chosen_sepsets = Set()
    Dict([v => 1.0 for v=1:length(clusters)])
    
    result = JoinTree(
        Graph(length(clusters)), 
        clusters, 
        Dict([v => eye(cluster_size(clusters[v], an)) for v=1:length(clusters)]),
        Dict()
    )
    for c1 = 1:length(clusters)
        for c2= 1:length(clusters)
            if c1 != c2
                push!(candidate_sepsets,(c1, c2))
            end
        end
    end
    candidate_sepsets = sort(candidate_sepsets, by=c -> sepset_comparator(clusters[c[1]], clusters[c[2]]))
    i = 1
    n = length(clusters)

    while length(chosen_sepsets) < n-1
        i1, i2 = candidate_sepsets[i]
        c1, c2 = clusters[i1], clusters[i2]
        sepset = intersect(c1, c2)
        if (trees[c1] != trees[c2]) && !any([sepset==s for s in chosen_sepsets])
            push!(chosen_sepsets, sepset)
            trees[c1] = trees[c2] = union(c1, c2)
#             println(c1, " ", c2, " ", sepset)
            add_edge!(result.graph, i1, i2)
            push!(result.edge_to_num, Set([i1, i2]) => eye(cluster_size(sepset, an)))
        end
        i += 1
    end
    result
end

make_join_tree (generic function with 1 method)

In [16]:
jt = make_join_tree(cliques, an)
jt.clusters

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = (::getfield(Main, Symbol("##21#26")){Array{Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1},1},AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}})(::Int64) at none:0
└ @ Main ./none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = make_join_tree(::Array{Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1},1}, ::AcausalNets.Struct

6-element Array{Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1},1}:
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:g, 2)], [0.8 0.0 0.0 0.0; 0.0 0.2 0.0 0.0; 0.0 0.0 0.1 0.0; 0.0 0.0 0.0 0.9]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:e, 2), Variable(:g, 2)], Variable[Variable(:h, 2)], [0.05 0.0 … 0.0 0.0; 0.0 0.95 … 0.0 0.0; … ; 0.0 0.0 … 0.95 0.0; 0.0 0.0 … 0.0 0.05])]
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,

In [17]:
# gplot(jt.graph, nodelabel=[join([string(v) for v in c]) for c in jt.clusters])

In [18]:
function parent_systems(an::AcausalNet, ds::DiscreteQuantumSystem)
    result = DiscreteQuantumSystem[]
    for p in parents(ds)
        parent_system = systems(an)[variable_to_node(an, p)]
        if !(parent_system in result)
            push!(result, parent_system)
        end
    end
    return result
end

parent_systems (generic function with 1 method)

In [19]:
function family(an::AcausalNet, dqs::DiscreteQuantumSystem)
    union(Set([dqs]), Set(parent_systems(an, dqs)))
end

family (generic function with 1 method)

In [20]:
print(jt.clusters[1])
jt.vertex_to_num[1]

AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:g, 2)], [0.8 0.0 0.0 0.0; 0.0 0.2 0.0 0.0; 0.0 0.0 0.1 0.0; 0.0 0.0 0.0 0.9]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:e, 2), Variable(:g, 2)], Variable[Variable(:h, 2)], [0.05 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.95 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.95 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.05 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.95 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.05 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.95 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.05])]

8×8 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [21]:
event(system::AbstractMatrix, e::AbstractMatrix) = (e * system * e) / trace(e * system)


event (generic function with 1 method)

In [22]:
AcausalNets.Systems.variables(cluster::Vector{DiscreteQuantumSystem}) = return [dqs.variables for dqs in cluster] 


In [23]:
[variables(c) for c in jt.clusters]

6-element Array{Array{Array{Variable,1},1},1}:
 [[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]]
 [[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
 [[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]]
 [[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
 [[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]
 [[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]]

In [24]:
family(an, an.systems[2])
[c for c=1:length(jt.clusters) if is_subset(family(an, an.systems[1]), Set(jt.clusters[c]))][1]

6

In [25]:
var_xD = Variable(:xd, 1)
s1 = DiscreteQuantumSystem([var_xD], eye(1))
s2 = DiscreteQuantumSystem([var_xD], eye(1))
set1 = Set([s1])
set2 = Set([s2])

s1 == s2

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0


false

In [26]:
shallowcopy(jt::JoinTree) = JoinTree(
        jt.graph, 
        jt.clusters, 
        deepcopy(jt.vertex_to_num), 
        deepcopy(jt.edge_to_num)
    )

shallowcopy (generic function with 1 method)

In [27]:
function initialize(jt::JoinTree, an::AcausalNet, assignments::Dict=Dict())
    jt = shallowcopy(jt)
    for v1 in systems(an)
#         println(v1.variables)
        parent_cluster_ind = [c for c=1:length(jt.clusters) if is_subset(family(an, v1), Set(jt.clusters[c]))][1]
        mul_elem = eye(1)
        multiplied_indices = [] 
        multiplied_dimensions = []
#         println(variables(v1), " chooses ", variables(jt.clusters[parent_cluster_ind]))
        
        for v2 in [v for v in systems(an) if v in jt.clusters[parent_cluster_ind]]
            factor = eye(1)
            if v1 == v2
                assignment_factor = eye(1)
                for p in parent_systems(an, v2)
                    p_index = system_to_node(an, p)
                    push!(multiplied_indices, p_index)
                    push!(multiplied_dimensions, ncategories(p))
                    assignment_factor = kron(assignment_factor, eye(ncategories(p)))
                end
                for v in variables(v2)
                    assignment = get(assignments, v, eye(ncategories(v)))
                    assignment_factor = kron(assignment_factor, assignment)
                end
                factor = event(v2.distribution, assignment_factor) * trace(v2.distribution)
                push!(multiplied_indices, system_to_node(an, v2))
                push!(multiplied_dimensions, ncategories(v2))
            elseif !in(v2, parent_systems(an, v1))
                factor = eye(ncategories(v2))
                push!(multiplied_indices, system_to_node(an, v2))
                push!(multiplied_dimensions, ncategories(v2))
            end
            mul_elem = kron(mul_elem, factor)
        end
#         println(variables(v1), " has chosen")
        sorted_multiplied_indices = sort(multiplied_indices)
        right_order = [findfirst(multiplied_indices, m) for m in sorted_multiplied_indices]
        
#         println([as.cpds[i].target for i in multiplied_indices])
#         println(multiplied_indices)
#         println([as.cpds[i].target for i in sorted_multiplied_indices])
#         println(sorted_multiplied_indices)
#         println(right_order)
#         println("-----")
        multiplied_dimensions = [multiplied_dimensions[r] for r in right_order]
#         mul_elem = permute_systems(mul_elem, multiplied_dimensions, right_order)

        previous_init = jt.vertex_to_num[parent_cluster_ind]
        jt.vertex_to_num[parent_cluster_ind] = star(mul_elem, previous_init)
    end
#     println()
    jt
end

initialize (generic function with 2 methods)

In [28]:
init_jt = initialize(jt, an);
# print(init_jt.cliques)
# real(init_jt.vertex_to_num[5])

  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
  likely near In[28]:1
in initialize at In[27]
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = initialize(::JoinTree, ::AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}, ::Dict{Any,Any}) at In[27]:6
└ @ Main ./In[27]:6
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, c

In [29]:
roCwA

4×4 Diagonal{Float64,Array{Float64,1}}:
 0.7   ⋅    ⋅    ⋅ 
  ⋅   0.3   ⋅    ⋅ 
  ⋅    ⋅   0.2   ⋅ 
  ⋅    ⋅    ⋅   0.8

In [30]:
# from this it would seem that currently node [a, d, c] 
# holds conditional probability ADCwD

a_factor = kron(kron(roA, eye(ncategories(sys_d))), eye(ncategories(sys_c)))
c_factor = kron(eye(ncategories(sys_d)), roCwA)
c_factor = permute_systems(c_factor, [2,2,2], [2,1,3])
ADCwD = star(c_factor, a_factor)
trace(ADCwD)
ptrace(ADCwD, [2,2,2], [1,2])

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).

2×2 Array{Float64,2}:
 0.9  0.0
 0.0  1.1

In [31]:
roBwA

4×4 Diagonal{Float64,Array{Float64,1}}:
 0.5   ⋅    ⋅    ⋅ 
  ⋅   0.5   ⋅    ⋅ 
  ⋅    ⋅   0.4   ⋅ 
  ⋅    ⋅    ⋅   0.6

In [32]:
# [a, b, d] holds ABDwA
b_factor = kron(roBwA, eye(ncategories(sys_d)))
d_factor = kron(eye(ncategories(sys_a)), roDwB)

ABDwA = star(b_factor, d_factor)
# trace(ABDwA)
ptrace(ABDwA, [2,2,2], [3])

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0


4×4 Array{Float64,2}:
 0.5  0.0  0.0  0.0
 0.0  0.5  0.0  0.0
 0.0  0.0  0.4  0.0
 0.0  0.0  0.0  0.6

In [33]:
init_jt.clusters

6-element Array{Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1},1}:
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:g, 2)], [0.8 0.0 0.0 0.0; 0.0 0.2 0.0 0.0; 0.0 0.0 0.1 0.0; 0.0 0.0 0.0 0.9]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:e, 2), Variable(:g, 2)], Variable[Variable(:h, 2)], [0.05 0.0 … 0.0 0.0; 0.0 0.95 … 0.0 0.0; … ; 0.0 0.0 … 0.95 0.0; 0.0 0.0 … 0.0 0.05])]
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,

This is a raw message pass

In [34]:
ABDwA = init_jt.vertex_to_num[6] #ABD
# ptrace(BDwA, [2,2,2], [2,3])

8×8 Array{Float64,2}:
 0.175  0.0    0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.075  0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.05  0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.2  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.14  0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.06  0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.06  0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.0   0.24

In [35]:
ADCwD = init_jt.vertex_to_num[5] #ADC
# ptrace(ACwD, [2,2,2], [2,3])
# trace(ADCwD)


8×8 Array{Float64,2}:
 0.9  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.5  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.9  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5

In [36]:
# message from ADC to ABD
ADwD = ptrace(ADCwD, [2,2,2], [3])

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [37]:
# fitting the message into ABD
ADBwBD = kron(ADwD, eye(ncategories(sys_b)))
ABDwBD = permute_systems(ADBwBD, [2,2,2], [1, 3, 2])

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0


8×8 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [38]:
ABD = star(ABDwA, ABDwBD)
# ptrace(ABD, [2,2,2], [2,3])
trace(ABD)
# ptrace(ABD, [2,2,2], [])

  likely near In[38]:3


1.0000000000000002

In [39]:
# message from ABD to ADC
AD = ptrace(ABD, [2,2,2], [2])
msg = unstar(AD,ADwD)

4×4 Array{Float64,2}:
 0.225  0.0    0.0  0.0
 0.0    0.275  0.0  0.0
 0.0    0.0    0.2  0.0
 0.0    0.0    0.0  0.3

In [40]:
# fitting the message into ADC
ADCwC = kron(msg, eye(ncategories(sys_c)))
trace(ADCwC)

┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = top-level scope at none:0
└ @ Core none:0
  likely near In[40]:3


2.0

In [41]:
ADC = star(ADCwD, ADCwC)
trace(ADC)


  likely near In[41]:2


1.0000000000000002

In [42]:
ptrace(ADC, [2,2,2], [1,3])

2×2 Array{Float64,2}:
 0.425  0.0  
 0.0    0.575

In [43]:
function single_message_pass(from_ind::Int, to_ind::Int, jt::JoinTree, an::AcausalNet)
    jt = shallowcopy(jt)
    if (from_ind, to_ind) in edges(jt.graph)
        println("message from ", variables(jt.clusters[from_ind]), " to ", variables(jt.clusters[to_ind]))
        cluster_from = jt.clusters[from_ind]
        cluster_to = jt.clusters[to_ind]
        sepset = intersect(cluster_from, cluster_to)
#         println(cluster_from, " ", sepset, " ", cluster_to)
        to_trace_out_sym = setdiff(cluster_from, sepset)
        to_trace_out_ind = [findfirst(cluster_from, s) for s in to_trace_out_sym]
#         println("trace_out ", to_trace_out_ind)
        from_variables_sizes = [ncategories(v) for v in cluster_from]
#         println(from_variables_sizes)
        cluster_from_num = jt.vertex_to_num[from_ind]
        old_sepset_num = jt.edge_to_num[Set([from_ind, to_ind])]
        new_sepset_num = ptrace(cluster_from_num, from_variables_sizes, to_trace_out_ind)
#         println(new_sepset_num)
#         println(old_sepset_num)
        
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        jt.edge_to_num[Set([from_ind, to_ind])] = new_sepset_num 
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        
        cluster_to_num = jt.vertex_to_num[to_ind]
        
#         message = event(old_sepset_num, new_sepset_num) 
        message = unstar(new_sepset_num, old_sepset_num)
        
        message_sym = Vector(sepset)
#         println(message_sym)
        for v in cluster_to
            if !in(v, message_sym)
                push!(message_sym, v)
                message = kron(message, eye(ncategories(v)))
            end
        end
#         println(message_sym)
        message_sorted_inds = [findfirst(message_sym, s) for s in systems(an) if s in message_sym]
#         println(message_sorted_inds) 
        message_dims = [ncategories(s) for s in message_sym]
        message_sorted = permute_systems(message, message_dims, message_sorted_inds )
        jt.vertex_to_num[to_ind] = star(cluster_to_num, message_sorted) #* trace(cluster_to_num)
#         jt.vertex_to_num[to_ind] = star(cluster_to_num, message_sorted) 
    end
    return jt
end

single_message_pass (generic function with 1 method)

In [44]:
[variables(c) for c in jt.clusters]

6-element Array{Array{Array{Variable,1},1},1}:
 [[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]]
 [[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
 [[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]]
 [[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
 [[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]
 [[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]]

In [45]:
init_jt.edge_to_num
init_jt.vertex_to_num[5]
#bcd
ptrace(init_jt.vertex_to_num[5], [2,2,2], [2,3])

2×2 Array{Float64,2}:
 2.0  0.0
 0.0  2.0

In [46]:
init_jt.vertex_to_num[6]
#abc
ptrace(init_jt.vertex_to_num[6], [2,2,2], [2,3])

2×2 Array{Float64,2}:
 0.5  0.0
 0.0  0.5

In [47]:
methods(variables)

# 3 methods for generic function "variables":
[1] variables(cluster::Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1}) in Main at In[22]:1
[2] variables(ds::AcausalNets.Systems.DiscreteSystem) in AcausalNets.Systems at /home/marcin/.julia/dev/AcausalNets/src/Systems/discrete_system.jl:28
[3] variables(dbn::AcausalNets.Structures.DiscreteBayesNet) in AcausalNets.Structures at /home/marcin/.julia/dev/AcausalNets/src/Structures/bayes_net.jl:33

In [48]:
# bcd -> abc
passed = single_message_pass(5, 6, init_jt, an);

message from Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]] to Array{Variable,1}[[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]]


┌ Warning: `findfirst(A, v)` is deprecated, use `something(findfirst(isequal(v), A), 0)` instead.
│   caller = #47 at none:0 [inlined]
└ @ Core ./none:0
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = single_message_pass(::Int64, ::Int64, ::JoinTree, ::AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}) at In[43]:34
└ @ Main ./In[43]:34
┌ Warning: `findfirst(A, v)` is deprecated, use `something(findfirst(isequal(v), A), 0)` instead.
│   caller = (::getfield(Main, Symbol("##48#51")))(::AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}) at none:0
└ @ Main ./none:0


In [49]:
passed.vertex_to_num[5]

8×8 Array{Float64,2}:
 0.9  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.5  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.9  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5

In [50]:
passed.vertex_to_num[6]

8×8 Array{Float64,2}:
 0.175  0.0    0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.075  0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.05  0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.2  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.14  0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.06  0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.06  0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.0   0.24

In [51]:
ptrace(passed.vertex_to_num[5], [2,2,2], [2, 3])

2×2 Array{Float64,2}:
 2.0  0.0
 0.0  2.0

In [52]:
# abc -> bcd 
passed_2 = single_message_pass(6, 5, passed, an);
passed_2.vertex_to_num[5]
ptrace(passed_2.vertex_to_num[5], [2,2,2], [1, 3])

message from Array{Variable,1}[[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]] to Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]


2×2 Array{Float64,2}:
 0.425  0.0  
 0.0    0.575

In [53]:
function collect_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree, an::AcausalNet)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
#             println(jt.clusters[cluster_ind], " ", jt.clusters[neighbor])
            jt, cluster_marks = collect_evidence(neighbor, cluster_marks, jt, an)
            jt = single_message_pass(neighbor, cluster_ind, jt, an)
            # pass a message from neighbor to cluster_ind
        end
        
    end
    jt = shallowcopy(jt)
    jt, cluster_marks
    
end

collect_evidence (generic function with 1 method)

In [54]:
function distribute_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree, an::AcausalNet)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt = single_message_pass(cluster_ind, neighbor, jt, an)
            # pass a message from cluster_ind to neighbor
        end
    end
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt, cluster_mars = distribute_evidence(neighbor, cluster_marks, jt, an)
        end
    end
    jt = shallowcopy(jt)
    jt, cluster_marks
end

distribute_evidence (generic function with 1 method)

In [55]:
function global_propagation(jt::JoinTree, an::AcausalNet)
    jt = shallowcopy(jt)
    cluster_marks = [true for c in jt.clusters]
    arbitrary_cluster_ind = 1
#     println("Collect evidence")
    jt, cluster_marks = collect_evidence(arbitrary_cluster_ind, cluster_marks, jt, an)
    cluster_marks = [true for c in jt.clusters]
#     println("Distribute evidence")
    jt, cluster_marks = distribute_evidence(arbitrary_cluster_ind, cluster_marks, jt, an)
    return jt
end

global_propagation (generic function with 1 method)

In [56]:
propagated_jt = global_propagation(init_jt, an)
# propagated_jt = global_propagation(propagated_jt, bn)

message from Array{Variable,1}[[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
message from Array{Variable,1}[[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]] to Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]
message from Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]] to Array{Variable,1}[[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]]
message from Array{Variable,1}[[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
mess

JoinTree({6, 5} undirected simple Int64 graph, Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1}[[DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:g, 2)], [0.8 0.0 0.0 0.0; 0.0 0.2 0.0 0.0; 0.0 0.0 0.1 0.0; 0.0 0.0 0.0 0.9]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:e, 2), Variable(:g, 2)], Variable[Variable(:h, 2)], [0.05 0.0 … 0.0 0.0; 0.0 0.95 … 0.0 0.0; … ; 0.0 0.0 … 0.95 0.0; 0.0 0.0 … 0.0 0.05])], [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), Discr

In [57]:
[variables(c) for c in propagated_jt.clusters]

6-element Array{Array{Array{Variable,1},1},1}:
 [[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]]
 [[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
 [[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]]
 [[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
 [[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]
 [[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]]

In [58]:
BCD = propagated_jt.vertex_to_num[5]

8×8 Array{Float64,2}:
 0.2835  0.0     0.0     0.0     0.0    0.0    0.0   0.0 
 0.0     0.0315  0.0     0.0     0.0    0.0    0.0   0.0 
 0.0     0.0     0.0675  0.0     0.0    0.0    0.0   0.0 
 0.0     0.0     0.0     0.0675  0.0    0.0    0.0   0.0 
 0.0     0.0     0.0     0.0     0.099  0.0    0.0   0.0 
 0.0     0.0     0.0     0.0     0.0    0.011  0.0   0.0 
 0.0     0.0     0.0     0.0     0.0    0.0    0.22  0.0 
 0.0     0.0     0.0     0.0     0.0    0.0    0.0   0.22

In [59]:
ptrace(BCD, [2,2,2], [1,3])

2×2 Array{Float64,2}:
 0.425  0.0  
 0.0    0.575

In [60]:
ABC = propagated_jt.vertex_to_num[6]

8×8 Array{Float64,2}:
 0.175  0.0    0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.075  0.0   0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.05  0.0  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.2  0.0   0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.14  0.0   0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.06  0.0   0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.06  0.0 
 0.0    0.0    0.0   0.0  0.0   0.0   0.0   0.24

In [61]:
ptrace(ABD, [2,2,2], [1,2])

2×2 Array{Float64,2}:
 0.425  0.0  
 0.0    0.575

In [62]:
# CDE
ptrace(propagated_jt.vertex_to_num[4], [2,2,2], [2,3])


2×2 Array{Float64,2}:
 0.425  0.0  
 0.0    0.575

# To sum everything up

In [63]:
an = example_an
# gplot(bn.dag, nodelabel=nodes(bn))

AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}({8, 9} directed simple Int64 graph, AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}[DiscreteSystem{AbstractArray{T,2} where T}(Variable[], Variable[Variable(:a, 2)], [0.5 0.0; 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:b, 2)], [0.5 0.0 0.0 0.0; 0.0 0.5 0.0 0.0; 0.0 0.0 0.4 0.0; 0.0 0.0 0.0 0.6]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:b, 2)], Variable[Variable(:d, 2)], [0.9 0.0 0.0 0.0; 0.0 0.1 0.0 0.0; 0.0 0.0 0.5 0.0; 0.0 0.0 0.0 0.5]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{Abstrac

In [64]:
moral_g = moral_graph(an)
tri_moral_graph, cliques = triangulate(moral_g, an)
jt = make_join_tree(cliques, an);

In [65]:
a = 0
a_oper = ketbra(a,a,2)
assignments = Dict([
        var_a => a_oper
        ])

Dict{Variable,Array{Complex{Float64},2}} with 1 entry:
  Variable(:a, 2) => Complex{Float64}[1.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im]

In [66]:
initialized_jt = initialize(jt, an, assignments)
propagated_jt = global_propagation(initialized_jt, an)
propagated_jt.clusters

  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
  likely near In[66]:1
in initialize at In[27]
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix(1.0I, m, m)` or `Matrix{Float64}(I, m, m)`. If `Float64` element type is not necessary, consider the shorter `Matrix(I, m, m)` (with default `eltype(I)` `Bool`).
│   caller = initialize(::JoinTree, ::AcausalNets.Structures.DiscreteBayesNet{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T}}, ::Dict{Variable,Array{Complex{Float64},2}}) at In[27]:6
└ @ Main ./In[27]:6
┌ Warning: `eye(m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. 

message from Array{Variable,1}[[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
message from Array{Variable,1}[[Variable(:a, 2)], [Variable(:b, 2)], [Variable(:c, 2)]] to Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]]
message from 

┌ Warning: `sqrtm` is deprecated, use `sqrt` instead.
│   caller = star(::Array{Float64,2}, ::Array{Complex{Float64},2}) at math.jl:11
└ @ AcausalNets.Common /home/marcin/.julia/dev/AcausalNets/src/Common/math.jl:11
┌ Warning: `findfirst(A, v)` is deprecated, use `something(findfirst(isequal(v), A), 0)` instead.
│   caller = system_to_node at bayes_net.jl:30 [inlined]
└ @ Core /home/marcin/.julia/dev/AcausalNets/src/Structures/bayes_net.jl:30


Array{Variable,1}[[Variable(:b, 2)], [Variable(:c, 2)], [Variable(:d, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]] to Array{Variable,1}[[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]]
message from Array{Variable,1}[[Variable(:e, 2)], [Variable(:g, 2)], [Variable(:h, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:e, 2)], [Variable(:g, 2)]] to Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]]
message from Array{Variable,1}[[Variable(:c, 2)], [Variable(:d, 2)], [Variable(:e, 2)]] to Array{Variable,1}[[Variable(:d, 2)], [Variable(:e, 2)], [Variable(:f, 2)]]
message from Arra

┌ Warning: `sqrtm` is deprecated, use `sqrt` instead.
│   caller = star(::Array{Complex{Float64},2}, ::Array{Complex{Float64},2}) at math.jl:11
└ @ AcausalNets.Common /home/marcin/.julia/dev/AcausalNets/src/Common/math.jl:11
┌ Warning: `sqrtm` is deprecated, use `sqrt` instead.
│   caller = star(::Array{Complex{Float64},2}, ::Array{Complex{Float64},2}) at math.jl:11
└ @ AcausalNets.Common /home/marcin/.julia/dev/AcausalNets/src/Common/math.jl:11


6-element Array{Array{AcausalNets.Systems.DiscreteSystem{AbstractArray{T,2} where T},1},1}:
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:g, 2)], [0.8 0.0 0.0 0.0; 0.0 0.2 0.0 0.0; 0.0 0.0 0.1 0.0; 0.0 0.0 0.0 0.9]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:e, 2), Variable(:g, 2)], Variable[Variable(:h, 2)], [0.05 0.0 … 0.0 0.0; 0.0 0.95 … 0.0 0.0; … ; 0.0 0.0 … 0.95 0.0; 0.0 0.0 … 0.0 0.05])]
 [DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:a, 2)], Variable[Variable(:c, 2)], [0.7 0.0 0.0 0.0; 0.0 0.3 0.0 0.0; 0.0 0.0 0.2 0.0; 0.0 0.0 0.0 0.8]), DiscreteSystem{AbstractArray{T,2} where T}(Variable[Variable(:c, 2)], Variable[Variable(:e, 2)], [0.3 0.0 0.0 0.0; 0.0 0.7 0.0 0.0; 0.0 0.0 0.6 0.0; 0.0 0.0 0.0 0.4]), DiscreteSystem{AbstractArray{T,

In [67]:
aBC = propagated_jt.vertex_to_num[6]
ptrace(aBC, [2,2,2], [2,3])

2×2 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im